In [1]:
!pwd

/home/yutankim/Desktop/EfficientDet_TensorFlow2


In [2]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing
from data.dataloader import DataLoader

def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

#모델 구조
model = EfficientDet()
print_model_summary(model)

#weight 불러오기
load_weights_from_epoch = 49
model.load_weights(filepath=Config.best_model_dir+"epoch-{}".format(load_weights_from_epoch))
#model.load_weights(filepath=Config.save_model_dir+"saved_model")

Model: "efficient_det_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficient_net_1 (EfficientNe multiple                  11704004  
_________________________________________________________________
bi_fpn_1 (BiFPN)             multiple                  283153    
_________________________________________________________________
box_class_predict_1 (BoxClas multiple                  456501    
Total params: 12,443,658
Trainable params: 12,377,914
Non-trainable params: 65,744
_________________________________________________________________


In [3]:
import numpy  as np
from PIL import Image
import os

# path = './data/fire_smoke/train/JPEGImages'
# count = 0
# image_list = []
# for item in os.listdir(path):
#     imgpath = path +'/' + item
#     img = Image.open(imgpath)
#     arr = np.array(img)
#     image_list.append(arr)
#     #img.save(str(count) +  "output.png")
#     count = count +1
# print(np.array(image_list).shape)

# for image in tf.data.Dataset.from_tensor_slices(image_list).batch(1).take(500):
#     image = tf.image.resize(image, (640, 640))
#     print(image.shape)

In [4]:
# 여기 부분, 기존 코드의 data_preprocesssing 과 동일하게 맞추기
# 배치사이즈 ( 32 ) 에 동일하게 맞춰서 넣어줘야한다.

# def representative_data_gen():
#     dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
#     for i in range(100):
#         image = next(iter(dataset_list))
#         image = tf.io.read_file(image)
#         image = tf.io.decode_jpeg(image, channels=3)
#         image = tf.image.resize(image, (512,512))
#         #image = tf.cast(image / 255., tf.float32)
#         image = tf.expand_dims(image, 0)
#         print(image.shape)
# # Model has only one input so each data point has one element
#         yield [image]
    
# def representative_data_gen():
#     dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
#     image_data = []
#     for dir in dataset_list:
        
    
#     for j in range(10):
#         image_temp = [ ]
#         for temp_dir in dataset_list.take(32):
#             image = tf.io.read_file(temp_dir)
#             image = tf.io.decode_jpeg(image, channels=3)
#             image = tf.image.resize(image, (512,512))
#             #image = tf.cast(image / 255., tf.float32)
#             image = tf.expand_dims(image, 0)
#             image_temp.append(image)
#     # Model has only one input so each data point has one element
#         yield [image_temp]
        
def representative_data_gen():
    path = './data/fire_smoke/train/JPEGImages'
    count = 0
    image_list = []
    for item in os.listdir(path):
        imgpath = path +'/' + item
        img = Image.open(imgpath)
        arr = np.array(img)
        image_list.append(arr)
        #img.save(str(count) +  "output.png")
        count = count +1

    for image in tf.data.Dataset.from_tensor_slices(image_list).batch(1).take(500):
        image = tf.image.resize(image, (640, 640))
        yield [image]
            
#     #### 수정좀 해보기
# def representative_data_gen():
#   dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
#   for i in range(100):
#     image = next(iter(dataset_list))
#     image = tf.io.read_file(image)
#     image = tf.io.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (512,512))
#     # image = tf.cast(image / 255., tf.float32)
#     image = tf.expand_dims(image, 0)
# # Model has only one input so each data point has one element
#     yield [image]

In [5]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model_quant = converter.convert()

2021-11-03 20:52:27.970053: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-11-03 20:52:27.970216: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-11-03 20:52:27.972909: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:3b:00.0, compute capability: 7.5
2021-11-03 20:52:28.083241: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.018ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2021-11-03 20:52:32.365469: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-11-03 20:52:32.365553: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuf

In [6]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)

In [7]:
#interpreter.resize_tensor_input(0, [ 32 , 512, 512,   3])
interpreter.allocate_tensors()
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

name : input_1
index : 0
shape : [  1 640 640   3]
shape_signature : [  1 640 640   3]
dtype : <class 'numpy.uint8'>
quantization : (1.0, 0)
quantization_parameters : {'scales': array([1.], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/efficient_net_1/sequential_10/mb_conv_31/se_block_31/global_average_pooling2d_31/Mean/reduction_indices
index : 1
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/bi_fpn_1/bi_fpn_module_3/up_sampling2d_12/mul
index : 2
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

na

index : 78
shape : [240]
shape_signature : [240]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.14992766, 0.17168996, 0.15227245, 0.14718863, 0.14188062,
       0.14542964, 0.16811746, 0.16802853, 0.13613182, 0.15196522,
       0.1555083 , 0.1427638 , 0.16341192, 0.15410094, 0.14930907,
       0.18233041, 0.16767015, 0.13878109, 0.15409042, 0.14958158,
       0.14970149, 0.14202629, 0.1545134 , 0.15276268, 0.15126444,
       0.14611539, 0.1478648 , 0.14470264, 0.15566714, 0.17027235,
       0.15028735, 0.14469643, 0.16103613, 0.15294024, 0.16036534,
       0.13840935, 0.16590035, 0.14690535, 0.14853458, 0.16191486,
       0.16506626, 0.15391243, 0.16805921, 0.16102742, 0.14197062,
       0.14498545, 0.15641503, 0.14682667, 0.14887758, 0.15224263,
       0.15445308, 0.15074195, 0.14932713, 0.1537713 , 0.15421359,
       0.15302235, 0.18105783, 0.15632597, 0.15870312, 0.15326968,
       0.15046968, 0.15391435, 0.16816476, 0.14865506, 0

quantization_parameters : {'scales': array([1.9618108, 2.9067435, 2.7819111, 1.9209443, 2.4516795, 1.5928626,
       2.839938 , 2.3692744, 3.0223513, 2.4433758, 2.6884427, 1.878409 ,
       3.2389371, 2.6115363, 2.2178705, 1.9820688, 3.3347929, 3.2650096,
       1.6148642, 2.8538718, 3.5703347, 1.9581513, 3.0499449, 2.5456438,
       1.92694  , 2.48555  , 3.1051776, 2.3164237, 2.7624552, 2.1258397,
       2.7707522, 1.6265565, 2.8868687, 2.1166198, 2.7120214, 2.9222476,
       1.9614106, 2.6627994, 2.06389  , 2.8802505, 2.1190038, 2.4400783,
       2.9358573, 2.3677113, 1.7907764, 2.8580298, 2.3750668, 1.7766792,
       2.1098683, 3.1510508, 3.6566305, 3.0573153, 2.2619464, 3.1039636,
       2.4007936, 2.8496737, 2.4968998, 2.7267194, 2.4530025, 3.1440942,
       2.4553337, 2.3487544, 2.1321683, 2.3483696, 2.7167537, 1.7621949,
       3.0579455, 2.2727566, 2.1589992, 2.3544486, 2.2868698, 2.7013087,
       2.8162534, 2.1147127, 2.6273022, 2.6454918, 2.9940012, 1.729621 ,
       1.84013

quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.00395999, 0.00304009, 0.00502546, 0.00487439, 0.00313369,
       0.00260494, 0.00458797, 0.00556446, 0.00534079, 0.00450882,
       0.00449177, 0.00487532, 0.00464921, 0.00550743, 0.00533119,
       0.00515022, 0.00680666, 0.00461893, 0.00424966, 0.00508616,
       0.00337866, 0.00644961, 0.0045415 , 0.00466146, 0.00535376,
       0.00411194, 0.00465585, 0.00315084, 0.00424706, 0.00338909,
       0.00445506, 0.00541406, 0.00413111, 0.00492244, 0.00493582,
       0.00399993, 0.00581303, 0.00410044, 0.0044808 , 0.00481304,
       0.00385398, 0.00392961, 0.00589431, 0.00335077, 0.00463322,
       0.00549725, 0.00390793, 0.00382428, 0.00414373, 0.0053465 ,
       0.00355363, 0.00359318, 0.00368955, 0.00431081, 0.00490707,
       0.00407405, 0.00231174, 0.00457743, 0.00430076, 0.00335454,
       0.00357431, 0.00394132, 0.00450634, 0.00446944, 0.00223693,
       0.00424333, 0.0030111 , 0.00551989, 0.00423986, 0.00347395,
 

shape : [1152]
shape_signature : [1152]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([3.323081 , 3.416282 , 3.1990051, ..., 2.8203819, 3.1733067,
       3.0592775], dtype=float32), 'zero_points': array([0, 0, 0, ..., 0, 0, 0], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/efficient_net_1/sequential_12/mb_conv_40/batch_normalization_158/FusedBatchNormV3;efficient_det_1/efficient_net_1/sequential_12/mb_conv_40/depthwise_conv2d_40/depthwise;efficient_det_1/efficient_net_1/sequential_13/mb_conv_44/se_block_44/conv2d_193/Conv2D
index : 194
shape : [   1    5    5 1152]
shape_signature : [   1    5    5 1152]
dtype : <class 'numpy.int8'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([0.00424208, 0.00463822, 0.00345241, ..., 0.00359908, 0.00387151,
       0.00310717], dtype=float32), 'zero_points': array([0, 0, 0, ..., 0, 0, 0], dtype=int32), 'quantized_dimension': 3}
sparsity_p

...

quantization_parameters : {'scales': array([790.52563], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/efficient_net_1/sequential_9/mb_conv_29/batch_normalization_124/FusedBatchNormV3;efficient_det_1/efficient_net_1/sequential_10/mb_conv_31/se_block_31/conv2d_141/Conv2D;efficient_det_1/efficient_net_1/sequential_9/mb_conv_29/conv2d_131/Conv2D1
index : 543
shape : [  1  80  80 240]
shape_signature : [  1  80  80 240]
dtype : <class 'numpy.int8'>
quantization : (902.9962158203125, 12)
quantization_parameters : {'scales': array([902.9962], dtype=float32), 'zero_points': array([12], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/efficient_net_1/sequential_9/mb_conv_29/Sigmoid
index : 544
shape : [  1  80  80 240]
shape_signature : [  1  80  80 240]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625],

sparsity_parameters : {}

name : efficient_det_1/efficient_net_1/sequential_11/mb_conv_38/Sigmoid
index : 686
shape : [  1  20  20 672]
shape_signature : [  1  20  20 672]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/efficient_net_1/sequential_11/mb_conv_38/mul
index : 687
shape : [  1  20  20 672]
shape_signature : [  1  20  20 672]
dtype : <class 'numpy.int8'>
quantization : (1266.7725830078125, -128)
quantization_parameters : {'scales': array([1266.7726], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/efficient_net_1/sequential_11/mb_conv_38/batch_normalization_152/FusedBatchNormV3;efficient_det_1/efficient_net_1/sequential_12/mb_conv_39/batch_normalization_155/FusedBatchNormV3;efficient_det_

shape : [ 1 80 80 88]
shape_signature : [ 1 80 80 88]
dtype : <class 'numpy.int8'>
quantization : (1243.1185302734375, -128)
quantization_parameters : {'scales': array([1243.1185], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/bi_fpn_1/bi_fpn_module_3/weighted_feature_fusion_28/mul_1
index : 851
shape : [ 1 40 40 88]
shape_signature : [ 1 40 40 88]
dtype : <class 'numpy.int8'>
quantization : (460.6825256347656, -128)
quantization_parameters : {'scales': array([460.68253], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/bi_fpn_1/bi_fpn_module_3/weighted_feature_fusion_28/Sum/input
index : 852
shape : [ 3  1 40 40 88]
shape_signature : [ 3  1 40 40 88]
dtype : <class 'numpy.int8'>
quantization : (460.6825256347656, -128)
quantization_parameters : {'scales': array([460.68253], dtype=float32), 'zero_points': array([-12

quantization_parameters : {'scales': array([8008.5312], dtype=float32), 'zero_points': array([127], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/box_class_predict_1/Sigmoid
index : 1021
shape : [ 1 80 80  9]
shape_signature : [ 1 80 80  9]
dtype : <class 'numpy.int8'>
quantization : (0.00390625, -128)
quantization_parameters : {'scales': array([0.00390625], dtype=float32), 'zero_points': array([-128], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/box_class_predict_1/Sigmoid1
index : 1022
shape : [ 1 80 80  9]
shape_signature : [ 1 80 80  9]
dtype : <class 'numpy.int8'>
quantization : (6988.58837890625, -18)
quantization_parameters : {'scales': array([6988.5884], dtype=float32), 'zero_points': array([-18], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : efficient_det_1/box_class_predict_1/Reshape_1
index : 1023
shape : [    1 57600     1]
shape_signature : [    1 57600     1]


shape_signature : [4]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : 
index : 1183
shape : [2]
shape_signature : [2]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : 
index : 1184
shape : [240]
shape_signature : [240]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
sparsity_parameters : {}

name : 
index : 1185
shape : [4]
shape_signature : [4]
dtype : <class 'numpy.int32'>
quantization : (0.0, 0)
quantization_parameters : {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}
spa

In [8]:
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.float32'>


In [9]:
with open('./tflite_model_quant_D1_SGD.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [10]:
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print("-"*50)
print(output_details)

{'name': 'input_1', 'index': 0, 'shape': array([  1, 640, 640,   3], dtype=int32), 'shape_signature': array([  1, 640, 640,   3], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (1.0, 0), 'quantization_parameters': {'scales': array([1.], dtype=float32), 'zero_points': array([0], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
--------------------------------------------------
{'name': 'Identity', 'index': 1157, 'shape': array([    1, 76725,     5], dtype=int32), 'shape_signature': array([    1, 76725,     5], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
